In [ ]:
import json
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr

In [ ]:
corona_df=pd.read_csv('../data/raw/corona/de_corona.csv',sep='\t')

with open('../data/raw/metadata/de_metadata.json','r', encoding="utf8") as f:
    country_metadata=json.load(f)

region_map = {country_metadata["country_metadata"][i]["covid_region_code"]: 
    country_metadata["country_metadata"][i]["iso3166-2_code"] for i in range(len(country_metadata["country_metadata"]))}
corona_df["iso3166-2"] = corona_df["region_code"].map(region_map)

population_map = {country_metadata["country_metadata"][i]["iso3166-2_code"]: 
    country_metadata["country_metadata"][i]["population"] for i in range(len(country_metadata["country_metadata"]))}

corona_df["population"] = corona_df["iso3166-2"].map(population_map)
corona_df["cases_pc"] = corona_df["confirmed_addition"] / corona_df["population"]

weather_df = pd.read_csv("../data/raw/weather/weather.csv")

weather_df["TemperatureAboveGround"] = weather_df["TemperatureAboveGround"] - 273.15
weather_df = weather_df[weather_df["iso3166-2"].str.startswith("DE")]

df = corona_df.merge(weather_df)

df

In [ ]:
#Selecting the features/the collected weatherdata
Xs = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround', 
    'Totalprecipitation', 'UVIndex', 'WindSpeed']

#Adding a constant to the dataframe and the Xs list
df = sm.add_constant(df)
Xs.append("const")

In [ ]:
est = sm.OLS(df["confirmed_addition"], df[Xs], hasconst = True).fit()


print(est.summary())

In [ ]:
#Selecting the features/the collected weatherdata
Xs = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround', 
    'Totalprecipitation', 'UVIndex', 'WindSpeed']

#Adding a constant to the dataframe and the Xs list
df = sm.add_constant(df)
Xs.append("const")

In [ ]:
log_est = sm.OLS(np.log(df["confirmed_addition"] + 1), df[Xs], hasconst = True).fit()


print(log_est.summary())

In [ ]:
#Selecting the features/the collected weatherdata
Xs = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround', 
    'Totalprecipitation', 'UVIndex', 'WindSpeed']

#Adding a constant to the dataframe and the Xs list
df = sm.add_constant(df)
Xs.append("const")

In [ ]:
est_pc = sm.OLS(np.log(df["cases_pc"] + 1), df[Xs], hasconst = True).fit()
print(est_pc.summary())

In [ ]:
regions = ["const",]

for region in set(df["iso3166-2"]):
    if region != "DE-BE":
        df[region] = (df["iso3166-2"] == region).astype(int)
        regions.append(region)
        Xs.append(region)

df

In [ ]:
est_pc = sm.OLS(np.log(df["cases_pc"] + 1), df[regions], hasconst = True).fit()
print(est_pc.summary())

In [ ]:
est_pc = sm.OLS(np.log(df["cases_pc"] + 1), df[Xs], hasconst = True).fit()
print(est_pc.summary())

In [ ]:
est_clust = sm.OLS(np.log(df["cases_pc"] + 1), df[Xs], hasconst = True).fit(cov_type = "cluster", cov_kwds ={"groups": df["iso3166-2"]}, use_t = True)

print(est_clust.summary())